In [96]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt

def importarPQR():
    datos = pd.read_excel('FP.xlsx', header=2, nrows=5)
    return datos

def importarTiempos():
    datos = pd.read_excel('FP.xlsx', header=15, nrows=10)
    return datos

def importarTiemposAlisto(numeroMaquina):
    datos = pd.read_excel('FP.xlsx',sheet_name=1, header=14*(numeroMaquina-1)+4, nrows=10)
    return datos

datosExcel = importarPQR()
datosTiempos = importarTiempos()

ultimoTiempo = 0
tiempoGlobal = 0
contadorTiempo = [0, 0, 0, 0, 0]

colaMaquina1 = []
colaMaquina2 = []
colaMaquina3 = []
colaMaquina4 = []
colaMaquina5 = []

maquinas = [colaMaquina1, colaMaquina2, colaMaquina3, colaMaquina4, colaMaquina5]


def iniciarCromosoma():
    return np.array([10, 3, 7, 4, 5])

secuenciaPrincipal = iniciarCromosoma()


def asignarRestantes():
    global datosExcel
    global maquinas
    global contadorTiempo
    faltar = [1, 2, 6, 8, 9]
    for trabajo in faltar:
        variableExcel = 'J' + str(trabajo)
        maquina = datosExcel[variableExcel][0]
        maquinas[maquina-1] += [trabajo]
    tiempos = tiemposEnCola(secuenciaPrincipal, extraerElementosMaquinas())
    contador = 0
    for t in tiempos:
        contadorTiempo[contador] += t
        contador += 1


def secuencia():
    global datosTiempos
    global maquinas
    global ultimoTiempo
    global tiempoGlobal
    global contadorTiempo
    for i in range(len(maquinas)):
        tiempos = auxiliarOrdenarTiempos(maquinas[i], secuenciaPrincipal[i], i+1)
        nuevaCola = ordenarPorTiempos(tiempos, maquinas[i])
        maquinas[i] = nuevaCola

    print(contadorTiempo)
    print(tiempoGlobal)
    print(maquinas)
    a = np.array(contadorTiempo)
    x = np.where(a > tiempoGlobal)
    x = a[x]
    indice = np.argmin(x)
    cola = auxiliarEncontrarCola(indice, secuenciaPrincipal[indice])
    maquinas[cola-1] += [secuenciaPrincipal[indice]]
    tiempoGlobal += contadorTiempo[indice] - ultimoTiempo
    ultimoTiempo = contadorTiempo[indice]
    secuenciaPrincipal[indice] = maquinas[indice][0]
    maquinas[indice] = maquinas[indice][1:]
    tiemp = tiemposEnCola(secuenciaPrincipal, extraerElementosMaquinas())[indice]
    contadorTiempo[indice] += tiemp
    #print(tiempoGlobal)
    #print(maquinas)
    #print(secuenciaPrincipal)


def extraerElementosMaquinas():
    listaResultado = []
    global maquinas
    for maq in maquinas:
        if maq != []:
            listaResultado += [maq[0]]
        else:
            listaResultado += [None]
    return listaResultado


def auxiliarEncontrarCola(posicionTrabajo, trabajo):
    global datosExcel
    j = 'J' + str(trabajo)
    proceso = datosExcel[j]
    contador = 0
    for cola in proceso:
        if posicionTrabajo+1 == cola:
            if contador != 5:
                return proceso[contador + 1]
            else:
                return None
        contador += 1
    return None

def auxiliarOrdenarTiempos(colasMaquina, proceso, numeroMaquina):
    global datosTiempos
    listaTiempos = []
    for procesoAux in colasMaquina:
        datosAlisto = importarTiemposAlisto(numeroMaquina)
        Ji = 'J' + str(proceso)
        Jj = procesoAux-1
        t1 = datosTiempos['M' + str(numeroMaquina)][procesoAux-1]
        t2 = datosAlisto[Ji][Jj]
        listaTiempos += [t1 + t2]
    return listaTiempos

def ordenarPorTiempos(listaTiempos, listaOrdenar):
    nuevaLista = []
    indices = np.argsort(listaTiempos)
    for i in indices:
        nuevaLista += [listaOrdenar[i]]
    return nuevaLista

def tiemposEnCola(secuencia, colas):
    tiemposAux = []
    contador = 0
    for elemento in colas:
        if elemento is not None:
            tiemposAux += auxiliarOrdenarTiempos([elemento], secuencia[contador], contador+1)
            contador += 1
        else:
            tiemposAux += [0]

    return tiemposAux

asignarRestantes()
secuencia()
secuencia()
secuencia()




[63, 26, 48, 40, 0]
0
[[1], [8, 9], [2], [6], []]
[63, 70, 48, 40, 0]
26
[[1], [9], [2], [6, 3], []]
[63, 70, 48, 107, 0]
40
[[1], [9], [2, 4], [3], []]
[63, 70, 93, 107, 0]
48
[[1], [9], [4], [3], [7]]
[63, 70, 93, 107, 0]
63
[[], [9], [4], [3], [10, 7]]


IndexError: list index out of range

In [79]:
a = np.array([63, 26, 48, 40, 0])
x = np.where(a > 0)
x = a[x]
x

array([63, 26, 48, 40])